**TCN**

**数据集：**
- Day 1-6: Train
- Day 7-10: Validation
- Day 11-13: Test

**预测目标：**
超短期预测（4h/15min）的风功率

In [1]:
import numpy as np

n_back = 200
n_out = 16
n_pre = n_out*15*2
n_feature = 2

train_day = 6
validation_day = 4
test_day = 3

# 神经网络参数
filter_nums = np.arange(10, 30, 5)   # filter数量
kernel_size = np.arange(4, 10, 2)    # kernel大小
batch_size = 32    # 训练批次大小
epochs = 50       # 训练epoch

# Preperation

In [2]:
from math import sqrt
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tcn.tcn import TCN
from keras.models import Sequential
from keras import layers

2023-01-12 00:46:09.313799: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 00:46:09.974979: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-12 00:46:09.975076: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-12 00:46:13.614549: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
dataset = pd.read_csv('../wind_preprocessed.csv', header=0, index_col=0).query('day<14')

In [4]:
data = dataset[['speed_moveavg', 'power_moveavg']].rename(columns={'power_moveavg':'power', 'speed_moveavg':'speed'})
print(data.head())

                       speed    power
date                                 
2015-10-01 00:00:00  0.39627  0.38065
2015-10-01 00:00:30  0.39592  0.36943
2015-10-01 00:01:00  0.39538  0.38529
2015-10-01 00:01:30  0.39579  0.38892
2015-10-01 00:02:00  0.39627  0.41220


In [5]:
values = data.values
values = values.astype('float32')

def series_to_supervised(data, n_in, n_out, colname, dropnan=True):
    n_vars = colname
    # n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in n_vars]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out, 15*2):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (j)) for j in n_vars]
        else:
            names += [('%s(t+%d)' % (j, i)) for j in n_vars]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# 构建成监督学习问题
reframed = series_to_supervised(values, n_back, n_pre, ['speed', 'power'])
# 丢弃我们不想预测的列
for i in range(0, n_pre, 15*2):
    if i == 0:
        colname = 'speed(t)'
    else:
        colname = f'speed(t+{i})'
    reframed.drop(colname, axis=1, inplace=True)
# print(reframed.head(5))

In [6]:
# 分割为训练集和测试集
values = reframed.values
n_train = train_day*24*60*2
n_validation = validation_day*24*60*2
n_test = test_day*24*60*2
train = values[:n_train, :]
validation = values[n_train:n_train+n_validation, :]
test = values[-n_test:, :]
# 分为输入输出
n_obs = n_back * n_feature
train_X, train_y = train[:, :n_obs], train[:, -n_out:]
validation_X, validation_y = validation[:, :n_obs], validation[:, -n_out:]
test_X, test_y = test[:, :n_obs], test[:, -n_out:]
# 重塑成3D格式 [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_back, n_feature))
validation_X = validation_X.reshape((validation_X.shape[0], n_back, n_feature))
test_X = test_X.reshape((test_X.shape[0], n_back, n_feature))
print(train_X.shape, train_y.shape)
print(validation_X.shape, validation_y.shape)
print(test_X.shape, test_y.shape)

(17280, 200, 2) (17280, 16)
(11520, 200, 2) (11520, 16)
(8640, 200, 2) (8640, 16)


# Training

In [8]:
for f in filter_nums:
    for k in kernel_size:
        print('---Start---')
        print(f'filter nums:{f}')
        print(f'kernel size:{k}')
        model = Sequential([
                layers.Input(shape=(train_X.shape[1], train_X.shape[2])),
                TCN(nb_filters=f, kernel_size=int(k), dilations=[1, 2, 4, 8]),
                layers.Dense(units=n_out)
            ])
        model.compile(optimizer='adam', loss='mse', metrics=['mse'])
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(validation_X, validation_y), verbose=1, shuffle=False)
        
        model.save_weights(f'tcn_{f}_{k}.h5')
        with open(f'tcn_{f}_{k}_hist.pickle', 'wb') as file_pi:
            pickle.dump(history.history, file_pi)
        print('---Finish---')


---Start---
filter nums:10
kernel size:4
Epoch 1/50
540/540 [==============================] - 59s 90ms/step - loss: 0.0376 - mse: 0.0376 - val_loss: 0.0340 - val_mse: 0.0340
Epoch 2/50
540/540 [==============================] - 47s 88ms/step - loss: 0.0325 - mse: 0.0325 - val_loss: 0.0280 - val_mse: 0.0280
Epoch 3/50
540/540 [==============================] - 47s 88ms/step - loss: 0.0330 - mse: 0.0330 - val_loss: 0.0216 - val_mse: 0.0216
Epoch 4/50
540/540 [==============================] - 41s 76ms/step - loss: 0.0318 - mse: 0.0318 - val_loss: 0.0188 - val_mse: 0.0188
Epoch 5/50
540/540 [==============================] - 47s 87ms/step - loss: 0.0301 - mse: 0.0301 - val_loss: 0.0163 - val_mse: 0.0163
Epoch 6/50
540/540 [==============================] - 47s 88ms/step - loss: 0.0276 - mse: 0.0276 - val_loss: 0.0156 - val_mse: 0.0156
Epoch 7/50
540/540 [==============================] - 49s 90ms/step - loss: 0.0267 - mse: 0.0267 - val_loss: 0.0144 - val_mse: 0.0144
Epoch 8/50
540/540 [=